# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-43886f389f-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Sport Workflow Series ("FIFA World Cup events explorative search") 

Consider the following exploratory information need:

> Explore the participants and winners of different editions of the FIFA World Cup

## Useful URIs for the current workflow


The following are given:

| IRI           | Description    | Role      |
| -----------   | -----------    |-----------|
| `wdt:P1647`   | subproperty    | predicate |
| `wdt:P31`     | instance of    | predicate |
| `wdt:P106`    | profession     | predicate | 
| `wdt:P279`    | subclass       | predicate |
| `wdt:P27`     | nationality    | predicate |
| `wdt:P1532`   | country of sport    | predicate |
| `wd:Q29`      | Spain           | node |
| `wd:Q42267`   | Spain national football team | node |
| `wd:Q19317`   | FIFA World Cup  | node |
| `wd:Q176883`  | 2010 FIFA World cup  | node |
| `wd:Q208401`  | 2010 FIFA World Cup Final | node      |
| `wd:Q6979593`  |national football team | node      |


Also consider

```
?a wdt:P31 wd:Q6979593
```

is the BGP to retrieve all the  **national footbal teams**


## Workload Goals

1. Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

2. Identify the BGP that connects a country with a national footbal team or as a location to a world cup

3. In which countries took place a FIFA world cup or world cup final? With how many participants?

4. Compare number of participants and winners of different world cups
 
   4.1 How many times each country participated in a world cup?
   
   4.2 Which country participated in the most finals? Which country won the most finals?
   
   4.3 Who participated in the most world cups among Italy, France, Spain, and Germany?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?a wdt:P31 wd:Q6979593 .

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1220')]


1

In [4]:
queryString="""
SELECT * {
  wd:Q19317 ?p ?o .
}

"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct/P7441'), ('o', 'wm')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('o', 'https://livedata.bibsys.no/authority/14068828')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('o', 'http://viaf.org/viaf/133874960')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('o', 'https://d-nb.info/gnd/1047989-2')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('o', 'https://id.loc.gov/authorities/names/n98090291')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/030q7')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8408'), ('o', 'http://kbpedia.org/kko/rc/WorldCupCompetition')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('o', '14068828')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('o', 'http://www.wikidata.org/entit

84

# Workload Goals
#### 1. Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

In [5]:
# | `wd:Q6979593`  |national football team | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

   ?p ?o wd:Q6979593 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5627659'), ('pname', 'Template:Infobox national football team'), ('o', 'http://www.wikidata.org/prop/direct/P1423'), ('oname', 'template has topic')]
[('p', 'http://www.wikidata.org/entity/Q23905105'), ('pname', "women's national association football team"), ('o', 'http://www.wikidata.org/prop/direct/P1889'), ('oname', 'different from')]
[('p', 'http://www.wikidata.org/entity/Q106588536'), ('pname', "women's national American football team"), ('o', 'http://www.wikidata.org/prop/direct/P1889'), ('oname', 'different from')]
[('p', 'http://www.wikidata.org/entity/Q3589950'), ('pname', 'Ivory Coast Olympic football team'), ('o', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
[('p', 'http://www.wikidata.org/entity/Q651714'), ('pname', 'Bulgaria national under-21 football team'), ('o', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
[('p', 'http://www.wikidata.org/entity/Q3873500'), ('pname', 'Norw

20

In [6]:
# | `wd:Q6979593`  |national football team | node      |
queryString = """
SELECT DISTINCT ?team ?teamname 
WHERE { 

   ?team wdt:P279 wd:Q6979593 .
    # Labels
    ?team <http://schema.org/name> ?teamname .

    FILTER regex(?teamname, "national football team", "i") 
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q134916'), ('teamname', 'Uruguay national football team')]
[('team', 'http://www.wikidata.org/entity/Q181190'), ('teamname', 'Bosnia and Herzegovina national football team')]
[('team', 'http://www.wikidata.org/entity/Q170327'), ('teamname', 'South Africa national football team')]
[('team', 'http://www.wikidata.org/entity/Q186243'), ('teamname', 'Iraq national football team')]
[('team', 'http://www.wikidata.org/entity/Q191406'), ('teamname', 'Israel national football team')]
[('team', 'http://www.wikidata.org/entity/Q14132'), ('teamname', 'North Korea national football team')]
[('team', 'http://www.wikidata.org/entity/Q175145'), ('teamname', 'Ivory Coast national football team')]
[('team', 'http://www.wikidata.org/entity/Q180187'), ('teamname', 'Honduras national football team')]
[('team', 'http://www.wikidata.org/entity/Q130582'), ('teamname', 'China national football team')]
[('team', 'http://www.wikidata.org/entity/Q194387'), ('teamna

20

In [7]:
# count the number of national teams
# | `wd:Q6979593`  |national football team | node      |
queryString = """
SELECT (COUNT (DISTINCT ?team) AS ?countteam)
WHERE { 

   ?team wdt:P279 wd:Q6979593 .
    # Labels
    ?team <http://schema.org/name> ?teamname .

    FILTER regex(?teamname, "national football team", "i") 
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('countteam', '53')]


1

In [8]:
# | `wd:Q6979593`  |national football team | node      |
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

   wd:Q6979593 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .

} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7139622'), ('oname', 'Category:National association football teams')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q15944511'), ('oname', 'association football team')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('oname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q937857'), ('oname', 'association football player')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1194951'), ('oname', 'national sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q23905105'), ('oname', 

9

In [9]:
# `wd:Q19317`   | FIFA World Cup  | node |

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q19317 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .



    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q101751'), ('oname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q131591'), ('oname', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q132664'), ('oname', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q253414'), ('oname', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q48432'), ('oname', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8437937'), ('oname', 'Category:FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/dire

40

In [10]:
# `wd:Q19317`   | FIFA World Cup  | node |

queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q19317 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .



    FILTER regex(?oname, "team", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q34542827'), ('oname', 'international association football national teams competition')]
[('p', 'http://www.wikidata.org/prop/direct/P8115'), ('pname', 'eligible recipient'), ('o', 'http://www.wikidata.org/entity/Q6979593'), ('oname', 'national association football team')]


2

### A list of all world cups

In [11]:
# list of world cups
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    wd:Q19317 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "has part", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q101751'), ('oname', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q131591'), ('oname', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q132664'), ('oname', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q48432'), ('oname', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q170645'), ('oname', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q176883'), ('oname', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P

20

In [12]:
# start time and end time of world cup 2034

# [('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q43215379'), ('oname', '2034 FIFA World Cup')]
queryString = """
SELECT ?start ?end 
WHERE { 

    wd:Q43215379 wdt:P580 ?start .
    wd:Q43215379 wdt:P582 ?end .
    # Labels
   # ?p <http://schema.org/name> ?pname .
    #FILTER regex(?pname, "has part", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('start', '2034-06-01T00:00:00Z'), ('end', '2034-09-01T00:00:00Z')]


1

In [13]:
# [('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q43215379'), ('oname', '2034 FIFA World Cup')]
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

    ?p ?o wd:Q43215379 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "has part", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q49929125'), ('pname', 'Category:2034 FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P301'), ('oname', "category's main topic")]
[('p', 'http://www.wikidata.org/entity/Q19317'), ('pname', 'FIFA World Cup'), ('o', 'http://www.wikidata.org/prop/direct/P527'), ('oname', 'has part')]


2

### Show all of the world cups.

In [14]:
queryString = """
SELECT ?worldcupname ?start ?end
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start .
    ?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname 
} 
ORDER BY ?start
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('start', '1930-07-13T00:00:00Z'), ('end', '1930-07-30T00:00:00Z')]
[('worldcupname', '1934 FIFA World Cup'), ('start', '1934-05-27T00:00:00Z'), ('end', '1934-06-10T00:00:00Z')]
[('worldcupname', '1938 FIFA World Cup'), ('start', '1938-06-04T00:00:00Z'), ('end', '1938-06-19T00:00:00Z')]
[('worldcupname', '1950 FIFA World Cup'), ('start', '1950-06-24T00:00:00Z'), ('end', '1950-07-16T00:00:00Z')]
[('worldcupname', '1954 FIFA World Cup'), ('start', '1954-06-16T00:00:00Z'), ('end', '1954-07-04T00:00:00Z')]
[('worldcupname', '1958 FIFA World Cup'), ('start', '1958-06-06T00:00:00Z'), ('end', '1958-06-29T00:00:00Z')]
[('worldcupname', '1962 FIFA World Cup'), ('start', '1962-05-30T00:00:00Z'), ('end', '1962-06-17T00:00:00Z')]
[('worldcupname', '1966 FIFA World Cup'), ('start', '1966-07-11T00:00:00Z'), ('end', '1966-07-30T00:00:00Z')]
[('worldcupname', '1970 FIFA World Cup'), ('start', '1970-05-31T00:00:00Z'), ('end', '1970-06-21T00:00:00Z')]
[(

25

### The number of world cups until now. 

In [15]:
# The list of world cups until now. 
queryString = """
SELECT ?worldcupname ?start ?end
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start .
    ?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname 
    FILTER(?end < "2022-02-15"^^xsd:dateTime).
} 
ORDER BY ?start
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('start', '1930-07-13T00:00:00Z'), ('end', '1930-07-30T00:00:00Z')]
[('worldcupname', '1934 FIFA World Cup'), ('start', '1934-05-27T00:00:00Z'), ('end', '1934-06-10T00:00:00Z')]
[('worldcupname', '1938 FIFA World Cup'), ('start', '1938-06-04T00:00:00Z'), ('end', '1938-06-19T00:00:00Z')]
[('worldcupname', '1950 FIFA World Cup'), ('start', '1950-06-24T00:00:00Z'), ('end', '1950-07-16T00:00:00Z')]
[('worldcupname', '1954 FIFA World Cup'), ('start', '1954-06-16T00:00:00Z'), ('end', '1954-07-04T00:00:00Z')]
[('worldcupname', '1958 FIFA World Cup'), ('start', '1958-06-06T00:00:00Z'), ('end', '1958-06-29T00:00:00Z')]
[('worldcupname', '1962 FIFA World Cup'), ('start', '1962-05-30T00:00:00Z'), ('end', '1962-06-17T00:00:00Z')]
[('worldcupname', '1966 FIFA World Cup'), ('start', '1966-07-11T00:00:00Z'), ('end', '1966-07-30T00:00:00Z')]
[('worldcupname', '1970 FIFA World Cup'), ('start', '1970-05-31T00:00:00Z'), ('end', '1970-06-21T00:00:00Z')]
[(

21

In [16]:
# The number of world cups until now. 
queryString = """
SELECT (COUNT (DISTINCT ?worldcup) AS ?countworldcup)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start .
    ?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname 
    FILTER(?end < "2022-02-15"^^xsd:dateTime).
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('countworldcup', '21')]


1

In [17]:
#[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q79859'), ('oname', '2014 FIFA World Cup')]
queryString = """
SELECT *
WHERE { 

    wd:Q79859 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "episode", "i") 
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('o', '32'), ('pname', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('o', 'http://www.wikidata.org/entity/Q43310'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('o', '64'), ('pname', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('o', 'http://www.wikidata.org/entity/Q6660743'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('o', 'http://www.wikidata.org/entity/Q155'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('o', '2014fifaworldcupbrazil'), ('pname', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('pname', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('o', '298900323'), ('pname', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'

60

In [18]:
#[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q79859'), ('oname', '2014 FIFA World Cup')]
queryString = """
SELECT *
WHERE { 

    wd:Q79859 ?p ?o .
    
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "match", "i")  .
   # FILTER regex(?pname, "participating", "i") .
   # FILTER regex(?pname, "participants", "i") .
   # FILTER regex(?pname, "winner", "i") .
   # FILTER regex(?pname, "goal", "i") .
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## ('o', '32'), ('pname', 'number of participants')]

Results
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('o', '64'), ('pname', 'number of matches played/races/starts')]


1

In [19]:
#[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q79859'), ('oname', '2014 FIFA World Cup')]
queryString = """
SELECT *
WHERE { 

    wd:Q79859 ?p ?o .
   # wd:Q79859 wdt:P1350 ?matches .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "participating", "i") .
   # FILTER regex(?pname, "participants", "i") .
   # FILTER regex(?pname, "winner", "i") .
   # FILTER regex(?pname, "goal", "i") .
   # FILTER regex(?pname, "final", "i") .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## P1132 ('o', '32'), ('pname', 'number of participants')]
# P3967 final event

Results
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q676899'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q212564'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q268208'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q134916'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q181190'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q175145'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('o', 'http://www.wikidata.org/entity/Q180187'), ('pname', 'participating team')]
[('p', 'http://www.wikidata.org/prop/direct/P19

10

In [20]:
#[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'),
#('o', 'http://www.wikidata.org/entity/Q79859'), ('oname', '2014 FIFA World Cup')]
queryString = """
SELECT ?finalteams
WHERE { 
     wd:Q79859 wdt:P3967 ?final .
     ?final wdt:P1923 ?finalteams .
    # Labels
   # ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    # ?final <http://schema.org/name> ?finalname .

} 
LIMIT 30
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## P1132 ('o', '32'), ('pname', 'number of participants')]
# P3967 final event

Results
[('finalteams', 'http://www.wikidata.org/entity/Q79800')]
[('finalteams', 'http://www.wikidata.org/entity/Q43310')]


2

In [21]:
# goals and the name of the player who scored in final games. 
queryString = """
SELECT *
WHERE { 
     wd:Q79859 wdt:P3967 ?final .
     #?final wdt:P1923 ?finalteams .
     # ?final wdt:P1363 ?scoredby .
     ?final ?p ?o .
    # Labels
   #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    # ?final <http://schema.org/name> ?finalname .

} 
LIMIT 30
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## P1132 ('o', '32'), ('pname', 'number of participants')]
# P3967 final event
## goal scored by: Q104454

Results
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct/P7494'), ('o', '1220140')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct/P1363'), ('o', 'http://www.wikidata.org/entity/Q104454')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct/P1652'), ('o', 'http://www.wikidata.org/entity/Q315234')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct/P7492'), ('o', '1635')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/0_r_3tc')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('o', 'http:/

24

In [22]:
# goals and the name of the player who scored in final games. 
queryString = """
SELECT ?finalteams ?scoredby
WHERE { 
     wd:Q79859 wdt:P3967 ?final .
     #?final wdt:P1923 ?finalteams .
     ?final wdt:P1363 ?scoredby .
     #?final ?p ?o .
    # Labels
   #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    # ?final <http://schema.org/name> ?finalname .

} 
LIMIT 30
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## P1132 ('o', '32'), ('pname', 'number of participants')]
# P3967 final event
## goal scored by: Q104454

Results
[('scoredby', 'http://www.wikidata.org/entity/Q104454')]


1

In [23]:
# goals and the name of the player who scored in final games. 
queryString = """
SELECT *
WHERE { 
     wd:Q79859 wdt:P3967 ?final .
     #?final wdt:P1923 ?finalteams .
     ?final wdt:P1363 ?scoredby .
     ?scoredby ?p ?o .
    # Labels
     ?p <http://schema.org/name> ?pname .
     ?o <http://schema.org/name> ?oname .
    # ?final <http://schema.org/name> ?finalname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

## winner P1346
## participating team P1923
## number of goals P1351
## P1132 ('o', '32'), ('pname', 'number of participants')]
# P3967 final event
## goal scored by: Q104454

Results
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('scoredby', 'http://www.wikidata.org/entity/Q104454'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q937857'), ('pname', 'occupation'), ('oname', 'association football player')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('scoredby', 'http://www.wikidata.org/entity/Q104454'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q82595'), ('pname', 'league'), ('oname', 'Bundesliga')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('scoredby', 'http://www.wikidata.org/entity/Q104454'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('o', 'http://www.wikidata.org/entity/Q22669'), ('pname', 'participant in'), ('oname', 'UEFA Euro 2012')]
[('final', 'http://www.wikidata.org/entity/Q15926885'), ('scoredby', 'http://www.wikidata.org/entity/Q104454'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('o', 'http://www.wiki

10

In [24]:
# the list of all teams in world cup 2014 brazil 
queryString = """
SELECT DISTINCT ?teams ?teamsname
WHERE { 

    # wd:Q79859 ?p ?o .
    # wd:Q79859 wdt:P1350 ?matches .
    wd:Q79859 wdt:P1923 ?teams . 
    # Labels
    #?p <http://schema.org/name> ?pname .
    ?teams <http://schema.org/name> ?teamsname .
    #?o <http://schema.org/name> ?oname .
   
   # FILTER regex(?pname, "participants", "i") .
   # FILTER regex(?pname, "winner", "i") .
   # FILTER regex(?pname, "goal", "i") .
   # FILTER regex(?pname, "final", "i") .
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('teams', 'http://www.wikidata.org/entity/Q676899'), ('teamsname', 'Italy national association football team')]
[('teams', 'http://www.wikidata.org/entity/Q212564'), ('teamsname', 'Colombia national football team')]
[('teams', 'http://www.wikidata.org/entity/Q268208'), ('teamsname', 'Australia national association football team')]
[('teams', 'http://www.wikidata.org/entity/Q134916'), ('teamsname', 'Uruguay national football team')]
[('teams', 'http://www.wikidata.org/entity/Q181190'), ('teamsname', 'Bosnia and Herzegovina national football team')]
[('teams', 'http://www.wikidata.org/entity/Q175145'), ('teamsname', 'Ivory Coast national football team')]
[('teams', 'http://www.wikidata.org/entity/Q180187'), ('teamsname', 'Honduras national football team')]
[('teams', 'http://www.wikidata.org/entity/Q194387'), ('teamsname', 'Costa Rica national football team')]
[('teams', 'http://www.wikidata.org/entity/Q47050'), ('teamsname', 'Netherlands national association football team')]
[(

32

In [25]:
# number of participants of world cup 2014, its winner, the number of matches, goals and the name of all teams
queryString = """
SELECT *
WHERE { 
    wd:Q79859 wdt:P1350 ?matches .
    #wd:Q79859 wdt:P1923 ?teams . 
    wd:Q79859 wdt:P1346 ?winner . 
    wd:Q79859 wdt:P1351 ?numgoal . 
    wd:Q79859 wdt:P1132 ?numparticipants .
    # Labels
    #?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)


Results
[('matches', '64'), ('winner', 'http://www.wikidata.org/entity/Q43310'), ('numgoal', '171'), ('numparticipants', '32'), ('winnername', 'Germany national association football team')]


1

## List of finals

In [26]:
# The list of world cups until now. 
queryString = """
SELECT ?worldcupname ?start ?end
WHERE { 
    wd:Q79859 wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    ?final wdt:P1363 ?scoredby .
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start .
    ?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname 
    FILTER(?end < "2022-02-15"^^xsd:dateTime).
} 
ORDER BY ?start
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('start', '1930-07-13T00:00:00Z'), ('end', '1930-07-30T00:00:00Z')]
[('worldcupname', '1930 FIFA World Cup'), ('start', '1930-07-13T00:00:00Z'), ('end', '1930-07-30T00:00:00Z')]
[('worldcupname', '1934 FIFA World Cup'), ('start', '1934-05-27T00:00:00Z'), ('end', '1934-06-10T00:00:00Z')]
[('worldcupname', '1934 FIFA World Cup'), ('start', '1934-05-27T00:00:00Z'), ('end', '1934-06-10T00:00:00Z')]
[('worldcupname', '1938 FIFA World Cup'), ('start', '1938-06-04T00:00:00Z'), ('end', '1938-06-19T00:00:00Z')]
[('worldcupname', '1938 FIFA World Cup'), ('start', '1938-06-04T00:00:00Z'), ('end', '1938-06-19T00:00:00Z')]
[('worldcupname', '1950 FIFA World Cup'), ('start', '1950-06-24T00:00:00Z'), ('end', '1950-07-16T00:00:00Z')]
[('worldcupname', '1950 FIFA World Cup'), ('start', '1950-06-24T00:00:00Z'), ('end', '1950-07-16T00:00:00Z')]
[('worldcupname', '1954 FIFA World Cup'), ('start', '1954-06-16T00:00:00Z'), ('end', '1954-07-04T00:00:00Z')]
[(

40

## Participates in finals 

In [27]:
# The list of world cups until now. + the final teams
queryString = """
SELECT DISTINCT ?worldcupname (GROUP_CONCAT(?finalteamsname ;separator = "; ") as ?finalteams)
WHERE { 
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    #?final wdt:P1363 ?scoredby .
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
              wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?final <http://schema.org/name> ?finalname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    #?scoredby <http://schema.org/name> ?scoredbyname .
    FILTER(?end < "2022-02-15"^^xsd:dateTime).
} 
ORDER BY ?start
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('finalteams', 'Uruguay national football team; Argentina national football team')]
[('worldcupname', '1934 FIFA World Cup'), ('finalteams', 'Italy national association football team; Czechoslovakia national association football team')]
[('worldcupname', '1938 FIFA World Cup'), ('finalteams', 'Hungary national association football team')]
[('worldcupname', '1950 FIFA World Cup'), ('finalteams', 'Brazil national football team; Uruguay national football team')]
[('worldcupname', '1954 FIFA World Cup'), ('finalteams', 'Hungary national association football team; Germany national association football team')]
[('worldcupname', '1958 FIFA World Cup'), ('finalteams', 'Brazil national football team; Sweden national association football team')]
[('worldcupname', '1962 FIFA World Cup'), ('finalteams', 'Czechoslovakia national association football team')]
[('worldcupname', '1966 FIFA World Cup'), ('finalteams', 'West Germany at the 1966 FIFA World

21

In [28]:
# # Number of times that spain was in finals. 
queryString = """
SELECT COUNT (?final) AS ?countfinal
WHERE { 
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    #?final wdt:P1363 ?scoredby .
    wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup wdt:P580 ?start .
    #?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    #?final <http://schema.org/name> ?finalname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    #?scoredby <http://schema.org/name> ?scoredbyname .
    # FILTER(?end < "2022-02-15"^^xsd:dateTime) .
    FILTER regex(?finalteamsname, "spain", "i") .
} 

"""

print("Results")
run_query(queryString)

Results
[('countfinal', '1')]


1

In [29]:
# # Number of times that spain was in finals. 
queryString = """
SELECT ?final ?finalname ?finalteamsname
WHERE { 
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    #?final wdt:P1363 ?scoredby .
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start .
    ?worldcup wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?final <http://schema.org/name> ?finalname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    #?scoredby <http://schema.org/name> ?scoredbyname .
    # FILTER(?end < "2022-02-15"^^xsd:dateTime) .
    FILTER regex(?finalteamsname, "spain", "i") .
} 

"""

print("Results")
run_query(queryString)

Results
[('final', 'http://www.wikidata.org/entity/Q208401'), ('finalname', '2010 FIFA World Cup Final'), ('finalteamsname', 'Spain national association football team')]


1

## Identify the BGP for connecting national football teams with the participations in the finals or in the world cups

In [30]:
queryString = """
SELECT ?worldcupname ?finalteamsname 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
                 wdt:P582 ?end ;
                 wdt:P1350 ?matches ;
                 wdt:P1923 ?finalteams ; 
                 wdt:P1346 ?winner ; 
                 wdt:P1351 ?numgoal ; 
                 wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
GROUP BY ?worldcupname
ORDER BY DESC(?start)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2018 FIFA World Cup'), ('finalteamsname', 'Australia at the 2018 FIFA World Cup')]
[('worldcupname', '2018 FIFA World Cup'), ('finalteamsname', 'Egypt at the 2018 FIFA World Cup\u200e')]
[('worldcupname', '2018 FIFA World Cup'), ('finalteamsname', 'Argentina at the 2018 FIFA World Cup')]
[('worldcupname', '2018 FIFA World Cup'), ('finalteamsname', 'Peru at the 2018 FIFA World Cup')]
[('worldcupname', '2018 FIFA World Cup'), ('finalteamsname', 'Morocco at the 2018 FIFA World Cup\u200e')]


5

## The number of participants of every world cup, its winner, the number of matches, goals.

In [31]:
queryString = """
SELECT ?worldcupname ?numparticipants ?matches ?winnername ?numgoal
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
         wdt:P582 ?end ;
         wdt:P1350 ?matches ;
         wdt:P1923 ?teams ; 
         wdt:P1346 ?winner ; 
         wdt:P1351 ?numgoal ; 
         wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
GROUP BY ?worldcupname
ORDER BY ASC(?start)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('numparticipants', '13'), ('matches', '18'), ('winnername', 'Uruguay national football team'), ('numgoal', '70')]
[('worldcupname', '1934 FIFA World Cup'), ('numparticipants', '16'), ('matches', '17'), ('winnername', 'Italy national association football team'), ('numgoal', '70')]
[('worldcupname', '1938 FIFA World Cup'), ('numparticipants', '15'), ('matches', '18'), ('winnername', 'Italy national association football team'), ('numgoal', '84')]
[('worldcupname', '1950 FIFA World Cup'), ('numparticipants', '13'), ('matches', '22'), ('winnername', 'Uruguay national football team'), ('numgoal', '88')]
[('worldcupname', '1954 FIFA World Cup'), ('numparticipants', '16'), ('matches', '26'), ('winnername', 'Germany national association football team'), ('numgoal', '140')]
[('worldcupname', '1958 FIFA World Cup'), ('numparticipants', '16'), ('matches', '35'), ('winnername', 'Brazil national football team'), ('numgoal', '126')]
[('worldcupname',

20

## Maximum number of goals that a world cup has in history of world cups.

In [32]:
queryString = """
SELECT (MAX(DISTINCT ?numgoal) AS ?maxNumgoal)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
                 wdt:P582 ?end ;
                 wdt:P1350 ?matches ;
                 wdt:P1923 ?teams ; 
                 wdt:P1346 ?winner ; 
                 wdt:P1351 ?numgoal ; 
                 wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 

"""

print("Results")
run_query(queryString)

Results
[('maxNumgoal', '171')]


1

In [33]:
queryString = """
SELECT ?worldcupname (Max(?numgoal) AS ?maxNumgoal)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
     ?worldcup wdt:P580 ?start ;
             wdt:P582 ?end ;
             wdt:P1350 ?matches ;
             wdt:P1923 ?teams ; 
             wdt:P1346 ?winner ; 
             wdt:P1351 ?numgoal ;
             wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
ORDER BY DESC (?maxNumgoal)
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1998 FIFA World Cup'), ('maxNumgoal', '171')]
[('worldcupname', '2014 FIFA World Cup'), ('maxNumgoal', '171')]
[('worldcupname', '2018 FIFA World Cup'), ('maxNumgoal', '169')]
[('worldcupname', '2002 FIFA World Cup'), ('maxNumgoal', '161')]
[('worldcupname', '2006 FIFA World Cup'), ('maxNumgoal', '147')]
[('worldcupname', '1982 FIFA World Cup'), ('maxNumgoal', '146')]
[('worldcupname', '2010 FIFA World Cup'), ('maxNumgoal', '145')]
[('worldcupname', '1994 FIFA World Cup'), ('maxNumgoal', '141')]
[('worldcupname', '1954 FIFA World Cup'), ('maxNumgoal', '140')]
[('worldcupname', '1986 FIFA World Cup'), ('maxNumgoal', '132')]
[('worldcupname', '1958 FIFA World Cup'), ('maxNumgoal', '126')]
[('worldcupname', '1990 FIFA World Cup'), ('maxNumgoal', '115')]
[('worldcupname', '1978 FIFA World Cup'), ('maxNumgoal', '102')]
[('worldcupname', '1974 FIFA World Cup'), ('maxNumgoal', '97')]
[('worldcupname', '1970 FIFA World Cup'), ('maxNumgoal', '95')]
[('worldcupname', '

21

## Identify the BGP that connects a country with a national football team or as a location to a world cup

In [34]:
# | `wd:Q42267`   | Spain national football team | node |
# | `wd:Q29`      | Spain           | node |
queryString = """
SELECT *
WHERE { 

    wd:Q42267 ?p wd:Q29 .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname,"", "i") 
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

# country P17
# country P1532

Results
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]


2

## Identify the BGP that connects a country with a national footbal team.

In [35]:
# | `wd:Q42267`   | Spain national football team | node |
# | `wd:Q29`      | Spain           | node |
queryString = """
SELECT DISTINCT ?teamsname ?countryname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P1923 ?teams .
    ?teams wdt:P17 ?country .
    # Labels
    ?teams <http://schema.org/name> ?teamsname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER regex(?pname,"", "i") 
} 
ORDER BY ?countryname
LIMIT 20

"""

print("Results")
run_query(queryString)


Results
[('teamsname', 'Algeria national football team'), ('countryname', 'Algeria')]
[('teamsname', 'Angola national football team'), ('countryname', 'Angola')]
[('teamsname', 'Argentina national football team'), ('countryname', 'Argentina')]
[('teamsname', 'Argentina at the 2018 FIFA World Cup'), ('countryname', 'Argentina')]
[('teamsname', 'Australia national association football team'), ('countryname', 'Australia')]
[('teamsname', 'Australia at the 2018 FIFA World Cup'), ('countryname', 'Australia')]
[('teamsname', 'Austria national association football team'), ('countryname', 'Austria')]
[('teamsname', 'Belgium national football team'), ('countryname', 'Belgium')]
[('teamsname', 'Bolivia national football team'), ('countryname', 'Bolivia')]
[('teamsname', 'Bosnia and Herzegovina national football team'), ('countryname', 'Bosnia and Herzegovina')]
[('teamsname', 'Brazil national football team'), ('countryname', 'Brazil')]
[('teamsname', 'Brazil at the 2018 FIFA World Cup\u200e'), (

20

## Identify the BGP that connects a country to a location of a world cup.

In [36]:
# | `wd:Q176883`  | 2010 FIFA World cup  | node |
queryString = """
SELECT *
WHERE { 

    wd:Q176883 ?p ?o .
               
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?pname, "location", "i") 
} 
LIMIT 30
"""

print("Results")
run_query(queryString)
# wdt:P276 stadium

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q163521'), ('pname', 'location'), ('oname', 'FNB Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q173559'), ('pname', 'location'), ('oname', 'Cape Town Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q187087'), ('pname', 'location'), ('oname', 'Mbombela Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q181439'), ('pname', 'location'), ('oname', 'Free State Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q258'), ('pname', 'location'), ('oname', 'South Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www.wikidata.org/entity/Q182081'), ('pname', 'location'), ('oname', 'Ellis Park Stadium')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('o', 'http://www

11

In [37]:
# | `wd:Q176883`  | 2010 FIFA World cup  | node |
queryString = """
SELECT DISTINCT ?country ?countryname
WHERE { 

    wd:Q176883 ?p ?o ;
                wdt:P17 ?country .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    #FILTER regex(?pname, "location", "i") 
} 
LIMIT 30
"""

print("Results")
run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q258'), ('countryname', 'South Africa')]


1

## The location of world cups. 

In [38]:
queryString = """
SELECT DISTINCT ?worldcupname ?countryname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P17 ?country ;
              wdt:P580 ?start ;
              wdt:P582 ?end .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY ?start
LIMIT 30
"""

print("Results")
run_query(queryString)


Results
[('worldcupname', '1930 FIFA World Cup'), ('countryname', 'Uruguay')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy')]
[('worldcupname', '1938 FIFA World Cup'), ('countryname', 'France')]
[('worldcupname', '1950 FIFA World Cup'), ('countryname', 'Brazil')]
[('worldcupname', '1954 FIFA World Cup'), ('countryname', 'Switzerland')]
[('worldcupname', '1958 FIFA World Cup'), ('countryname', 'Sweden')]
[('worldcupname', '1962 FIFA World Cup'), ('countryname', 'Chile')]
[('worldcupname', '1966 FIFA World Cup'), ('countryname', 'United Kingdom')]
[('worldcupname', '1970 FIFA World Cup'), ('countryname', 'Mexico')]
[('worldcupname', '1974 FIFA World Cup'), ('countryname', 'Germany')]
[('worldcupname', '1978 FIFA World Cup'), ('countryname', 'Argentina')]
[('worldcupname', '1982 FIFA World Cup'), ('countryname', 'Mexico')]
[('worldcupname', '1986 FIFA World Cup'), ('countryname', 'Mexico')]
[('worldcupname', '1990 FIFA World Cup'), ('countryname', 'Italy')]
[('worldcup

26

## Show the stadium that finals have been hold in every world cup. 

In [39]:
queryString = """
SELECT DISTINCT ?worldcupname ?countryname ?stadiumname ?stadiumcountryname 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    ?worldcup wdt:P17 ?country ;
              wdt:P580 ?start ;
              wdt:P582 ?end ;
              wdt:P276 ?stadium .
    ?stadium wdt:P17 ?stadiumcountry .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?stadium <http://schema.org/name> ?stadiumname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?stadiumcountry <http://schema.org/name> ?stadiumcountryname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY ?start
LIMIT 10
"""

print("Results")
run_query(queryString)

# wdt:P276 stadium

Results
[('worldcupname', '1930 FIFA World Cup'), ('countryname', 'Uruguay'), ('stadiumname', 'Estadio Centenario'), ('stadiumcountryname', 'Uruguay')]
[('worldcupname', '1930 FIFA World Cup'), ('countryname', 'Uruguay'), ('stadiumname', 'Estadio Gran Parque Central'), ('stadiumcountryname', 'Uruguay')]
[('worldcupname', '1930 FIFA World Cup'), ('countryname', 'Uruguay'), ('stadiumname', 'Estadio Pocitos'), ('stadiumcountryname', 'Uruguay')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy'), ('stadiumname', 'Stadio Giuseppe Meazza'), ('stadiumcountryname', 'Italy')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy'), ('stadiumname', 'Stadio Olimpico Grande Torino'), ('stadiumcountryname', 'Italy')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy'), ('stadiumname', 'Stadio Artemio Franchi (Florence)'), ('stadiumcountryname', 'Italy')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy'), ('stadiumname', "Stadio Renato Dall'

10

In [40]:
queryString = """
SELECT ?worldcupname ?countryname (GROUP_CONCAT(?stadiumname; separator=";") as ?stadium) 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    ?worldcup wdt:P17 ?country ;
              wdt:P580 ?start ;
              wdt:P582 ?end ;
              wdt:P276 ?stadium .
    ?stadium wdt:P17 ?stadiumcountry .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?stadium <http://schema.org/name> ?stadiumname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?stadiumcountry <http://schema.org/name> ?stadiumcountryname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY ?start
LIMIT 20
"""

print("Results")
run_query(queryString)

# wdt:P276 stadium

Results
[('worldcupname', '1930 FIFA World Cup'), ('countryname', 'Uruguay'), ('stadium', 'Estadio Centenario;Estadio Gran Parque Central;Estadio Pocitos;Estadio Centenario;Estadio Gran Parque Central;Estadio Pocitos')]
[('worldcupname', '1934 FIFA World Cup'), ('countryname', 'Italy'), ('stadium', "Stadio Giuseppe Meazza;Stadio Olimpico Grande Torino;Stadio Artemio Franchi (Florence);Stadio Renato Dall'Ara;Stadio Luigi Ferraris;Stadio Giuseppe Grezar;Stadio Nazionale PNF;Stadio Giorgio Ascarelli;Stadio Giuseppe Meazza;Stadio Olimpico Grande Torino;Stadio Artemio Franchi (Florence);Stadio Renato Dall'Ara;Stadio Luigi Ferraris;Stadio Giuseppe Grezar;Stadio Nazionale PNF;Stadio Giorgio Ascarelli")]
[('worldcupname', '1938 FIFA World Cup'), ('countryname', 'France'), ('stadium', 'Stade du Fort Carré;Stadium Municipal;Stade Chaban-Delmas;Stade Jules Deschaseaux;Stade Auguste Delaune;Parc des Princes;Stade Olympique Yves-du-Manoir;Stade Vélodrome;Stade de la Meinau;Stade Victor Boucquey')]


20

## Count the number of stadiums every world cup had.

In [41]:
queryString = """
SELECT ?worldcupname (COUNT (?stadium) AS ?countStadium)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    ?worldcup wdt:P17 ?country ;
              wdt:P580 ?start ;
              wdt:P582 ?end ;
              wdt:P276 ?stadium .
    ?stadium wdt:P17 ?stadiumcountry .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?stadium <http://schema.org/name> ?stadiumname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?stadiumcountry <http://schema.org/name> ?stadiumcountryname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY DESC (?countStadium)
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2002 FIFA World Cup'), ('countStadium', '80')]
[('worldcupname', '2018 FIFA World Cup'), ('countStadium', '24')]
[('worldcupname', '2006 FIFA World Cup'), ('countStadium', '24')]
[('worldcupname', '1958 FIFA World Cup'), ('countStadium', '24')]
[('worldcupname', '2014 FIFA World Cup'), ('countStadium', '24')]
[('worldcupname', '2010 FIFA World Cup'), ('countStadium', '22')]
[('worldcupname', '1998 FIFA World Cup'), ('countStadium', '20')]
[('worldcupname', '1982 FIFA World Cup'), ('countStadium', '17')]
[('worldcupname', '1934 FIFA World Cup'), ('countStadium', '16')]
[('worldcupname', '1966 FIFA World Cup'), ('countStadium', '16')]
[('worldcupname', '1950 FIFA World Cup'), ('countStadium', '12')]
[('worldcupname', '1954 FIFA World Cup'), ('countStadium', '12')]
[('worldcupname', '1990 FIFA World Cup'), ('countStadium', '12')]
[('worldcupname', '1986 FIFA World Cup'), ('countStadium', '12')]
[('worldcupname', '1938 FIFA World Cup'), ('countStadium', '10')]
[(

21

## In which countries took place a FIFA world cup or world cup final? With how many participants?

In [42]:
queryString = """
SELECT DISTINCT ?worldcupname ?numparticipants ?finalname ?finalcountryname 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams ;
            wdt:P17 ?finalcountry .
    ?worldcup wdt:P17 ?country ;
              wdt:P580 ?start ;
              wdt:P582 ?end ;
              wdt:P276 ?stadium ;
              wdt:P1132 ?numparticipants .
    ?stadium wdt:P17 ?stadiumcountry .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?country <http://schema.org/name> ?countryname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?stadium <http://schema.org/name> ?stadiumname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?stadiumcountry <http://schema.org/name> ?stadiumcountryname .
    ?final <http://schema.org/name> ?finalname .
    ?finalcountry <http://schema.org/name> ?finalcountryname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY ?start
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('numparticipants', '13'), ('finalname', '1930 FIFA World Cup Final'), ('finalcountryname', 'Uruguay')]
[('worldcupname', '1934 FIFA World Cup'), ('numparticipants', '16'), ('finalname', '1934 FIFA World Cup Final'), ('finalcountryname', 'Italy')]
[('worldcupname', '1938 FIFA World Cup'), ('numparticipants', '15'), ('finalname', '1938 FIFA World Cup Final'), ('finalcountryname', 'France')]
[('worldcupname', '1950 FIFA World Cup'), ('numparticipants', '13'), ('finalname', 'Uruguay v Brazil 1950'), ('finalcountryname', 'Brazil')]
[('worldcupname', '1954 FIFA World Cup'), ('numparticipants', '16'), ('finalname', '1954 FIFA World Cup Final'), ('finalcountryname', 'Switzerland')]
[('worldcupname', '1958 FIFA World Cup'), ('numparticipants', '16'), ('finalname', '1958 FIFA World Cup Final'), ('finalcountryname', 'Brazil')]
[('worldcupname', '1962 FIFA World Cup'), ('numparticipants', '16'), ('finalname', '1962 FIFA World Cup Final'), ('finalc

20

## Compare number of participants and winners of different world cups.

In [43]:
queryString = """
SELECT ?worldcupname ?numparticipants ?winnername
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
         wdt:P582 ?end ;
         wdt:P1350 ?matches ;
         wdt:P1923 ?teams ; 
         wdt:P1346 ?winner ; 
         wdt:P1351 ?numgoal ; 
         wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
GROUP BY ?worldcupname
ORDER BY ASC(?numparticipants)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1950 FIFA World Cup'), ('numparticipants', '13'), ('winnername', 'Uruguay national football team')]
[('worldcupname', '1930 FIFA World Cup'), ('numparticipants', '13'), ('winnername', 'Uruguay national football team')]
[('worldcupname', '1938 FIFA World Cup'), ('numparticipants', '15'), ('winnername', 'Italy national association football team')]
[('worldcupname', '1966 FIFA World Cup'), ('numparticipants', '16'), ('winnername', 'England national association football team')]
[('worldcupname', '1934 FIFA World Cup'), ('numparticipants', '16'), ('winnername', 'Italy national association football team')]
[('worldcupname', '1958 FIFA World Cup'), ('numparticipants', '16'), ('winnername', 'Brazil national football team')]
[('worldcupname', '1954 FIFA World Cup'), ('numparticipants', '16'), ('winnername', 'Germany national association football team')]
[('worldcupname', '1970 FIFA World Cup'), ('numparticipants', '16'), ('winnername', 'Brazil national football team')

20

In [44]:
queryString = """
SELECT DISTINCT ?worldcupname ?winnername ?winnercountryname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P1346 ?winner ;
            wdt:P580 ?start ;
            wdt:P582 ?end .
    ?winner wdt:P17 ?winnercountry .       
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?winner <http://schema.org/name> ?winnername .
    ?winnercountry <http://schema.org/name> ?winnercountryname .
    
} 
ORDER BY ?start
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('winnername', 'Uruguay national football team'), ('winnercountryname', 'Uruguay')]
[('worldcupname', '1934 FIFA World Cup'), ('winnername', 'Italy national association football team'), ('winnercountryname', 'Italy')]
[('worldcupname', '1938 FIFA World Cup'), ('winnername', 'Italy national association football team'), ('winnercountryname', 'Italy')]
[('worldcupname', '1950 FIFA World Cup'), ('winnername', 'Uruguay national football team'), ('winnercountryname', 'Uruguay')]
[('worldcupname', '1954 FIFA World Cup'), ('winnername', 'Germany national association football team'), ('winnercountryname', 'Germany')]
[('worldcupname', '1958 FIFA World Cup'), ('winnername', 'Brazil national football team'), ('winnercountryname', 'Brazil')]
[('worldcupname', '1962 FIFA World Cup'), ('winnername', 'Brazil national football team'), ('winnercountryname', 'Brazil')]
[('worldcupname', '1966 FIFA World Cup'), ('winnername', 'England national association

20

### How many times each country participated in a world cup?

In [45]:
queryString = """
SELECT DISTINCT ?team ?teamname 
WHERE { 

   ?team wdt:P279 wd:Q6979593 .
    # Labels
    ?team <http://schema.org/name> ?teamname .

    FILTER regex(?teamname, "national football team", "i") 
} 
LIMIT 20
"""
print("Results")
run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q134916'), ('teamname', 'Uruguay national football team')]
[('team', 'http://www.wikidata.org/entity/Q181190'), ('teamname', 'Bosnia and Herzegovina national football team')]
[('team', 'http://www.wikidata.org/entity/Q170327'), ('teamname', 'South Africa national football team')]
[('team', 'http://www.wikidata.org/entity/Q186243'), ('teamname', 'Iraq national football team')]
[('team', 'http://www.wikidata.org/entity/Q191406'), ('teamname', 'Israel national football team')]
[('team', 'http://www.wikidata.org/entity/Q14132'), ('teamname', 'North Korea national football team')]
[('team', 'http://www.wikidata.org/entity/Q175145'), ('teamname', 'Ivory Coast national football team')]
[('team', 'http://www.wikidata.org/entity/Q180187'), ('teamname', 'Honduras national football team')]
[('team', 'http://www.wikidata.org/entity/Q130582'), ('teamname', 'China national football team')]
[('team', 'http://www.wikidata.org/entity/Q194387'), ('teamna

20

In [46]:

queryString = """
SELECT ?teamsname (COUNT(DISTINCT ?worldcup) AS ?countworldcup)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P1923 ?teams .
    ?teams wdt:P17 ?teamscountry .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname 
    ?teams <http://schema.org/name> ?teamsname .
} 
ORDER BY DESC (?countworldcup)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('teamsname', 'Brazil national football team'), ('countworldcup', '20')]
[('teamsname', 'Italy national association football team'), ('countworldcup', '18')]
[('teamsname', 'Germany national association football team'), ('countworldcup', '18')]
[('teamsname', 'Argentina national football team'), ('countworldcup', '16')]
[('teamsname', 'Mexico national football team'), ('countworldcup', '15')]
[('teamsname', 'France national association football team'), ('countworldcup', '14')]
[('teamsname', 'Spain national association football team'), ('countworldcup', '14')]
[('teamsname', 'England national association football team'), ('countworldcup', '14')]
[('teamsname', 'Belgium national football team'), ('countworldcup', '12')]
[('teamsname', 'Uruguay national football team'), ('countworldcup', '12')]
[('teamsname', 'Sweden national association football team'), ('countworldcup', '11')]
[('teamsname', "United States men's national soccer team"), ('countworldcup', '10')]
[('teamsname', '

40

### Which country participated in the most finals? Which country won the most finals?
#### Who participated in the most world cups among Italy, France, Spain, and Germany?

In [47]:
queryString = """
SELECT ?finalteamscountryname (COUNT (?final) AS ?countfinal)
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .
    ?final wdt:P1923 ?finalteams .
    ?finalteams wdt:P17 ?finalteamscountry .
    # Labels

    ?worldcup <http://schema.org/name> ?worldcupname .
    ?finalteams <http://schema.org/name> ?finalteamsname .
    ?final <http://schema.org/name> ?finalname .
    ?finalteamscountry <http://schema.org/name> ?finalteamscountryname .
    #FILTER regex(?pname, "location", "i") 
} 
ORDER BY DESC (?countfinal)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('finalteamscountryname', 'Germany'), ('countfinal', '6')]
[('finalteamscountryname', 'Brazil'), ('countfinal', '5')]
[('finalteamscountryname', 'Italy'), ('countfinal', '4')]
[('finalteamscountryname', 'Argentina'), ('countfinal', '3')]
[('finalteamscountryname', 'Uruguay'), ('countfinal', '2')]
[('finalteamscountryname', 'Netherlands'), ('countfinal', '2')]
[('finalteamscountryname', 'France'), ('countfinal', '2')]
[('finalteamscountryname', 'Hungary'), ('countfinal', '2')]
[('finalteamscountryname', 'Czechoslovakia'), ('countfinal', '2')]
[('finalteamscountryname', 'United Kingdom'), ('countfinal', '2')]
[('finalteamscountryname', 'Spain'), ('countfinal', '1')]
[('finalteamscountryname', 'Sweden'), ('countfinal', '1')]


12

## How many world cups we had generally?

In [48]:
## how many world cups we have generally?
queryString = """
SELECT COUNT (?worldcup) AS ?countworldcup
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
              wdt:P582 ?end .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    FILTER(?end < "2022-02-15"^^xsd:dateTime) .
} 

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('countworldcup', '21')]


1

In [49]:
## how many finals we had during the history of world cup?(becuase some of the world cups were cancel because of world war.)
queryString = """
SELECT COUNT (?final) AS ?countfinal
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final .

} 

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('countfinal', '21')]


1

In [50]:
## how many finals we had during the history of world cup?
queryString = """
SELECT ?worldcupname ?finalname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3967 ?final ;
            wdt:P580 ?start ;
            wdt:P582 ?end .
    
        # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?final <http://schema.org/name> ?finalname .
} 
ORDER BY (?start)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1930 FIFA World Cup'), ('finalname', '1930 FIFA World Cup Final')]
[('worldcupname', '1934 FIFA World Cup'), ('finalname', '1934 FIFA World Cup Final')]
[('worldcupname', '1938 FIFA World Cup'), ('finalname', '1938 FIFA World Cup Final')]
[('worldcupname', '1950 FIFA World Cup'), ('finalname', 'Uruguay v Brazil 1950')]
[('worldcupname', '1954 FIFA World Cup'), ('finalname', '1954 FIFA World Cup Final')]
[('worldcupname', '1958 FIFA World Cup'), ('finalname', '1958 FIFA World Cup Final')]
[('worldcupname', '1962 FIFA World Cup'), ('finalname', '1962 FIFA World Cup Final')]
[('worldcupname', '1966 FIFA World Cup'), ('finalname', '1966 FIFA World Cup Final')]
[('worldcupname', '1970 FIFA World Cup'), ('finalname', '1970 FIFA World Cup Final')]
[('worldcupname', '1974 FIFA World Cup'), ('finalname', '1974 FIFA World Cup Final')]
[('worldcupname', '1978 FIFA World Cup'), ('finalname', '1978 FIFA World Cup Final')]
[('worldcupname', '1982 FIFA World Cup'), ('finaln

21

In [51]:
# which world cup had zero matches and it was cancelled? 
queryString = """
SELECT ?worldcupname ?matches ?worldcup
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P580 ?start ;
         wdt:P582 ?end ;
         wdt:P1350 ?matches ;
         wdt:P1923 ?teams ; 
         wdt:P1346 ?winner ; 
         wdt:P1351 ?numgoal ; 
         wdt:P1132 ?numparticipants .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .
    ?winner <http://schema.org/name> ?winnername .
} 
GROUP BY ?worldcupname
ORDER BY ASC(?matches)
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1934 FIFA World Cup'), ('matches', '17'), ('worldcup', 'http://www.wikidata.org/entity/Q1477177')]
[('worldcupname', '1938 FIFA World Cup'), ('matches', '18'), ('worldcup', 'http://www.wikidata.org/entity/Q131591')]
[('worldcupname', '1930 FIFA World Cup'), ('matches', '18'), ('worldcup', 'http://www.wikidata.org/entity/Q48432')]
[('worldcupname', '1950 FIFA World Cup'), ('matches', '22'), ('worldcup', 'http://www.wikidata.org/entity/Q132515')]
[('worldcupname', '1954 FIFA World Cup'), ('matches', '26'), ('worldcup', 'http://www.wikidata.org/entity/Q131075')]
[('worldcupname', '1970 FIFA World Cup'), ('matches', '32'), ('worldcup', 'http://www.wikidata.org/entity/Q132664')]
[('worldcupname', '1966 FIFA World Cup'), ('matches', '32'), ('worldcup', 'http://www.wikidata.org/entity/Q134202')]
[('worldcupname', '1962 FIFA World Cup'), ('matches', '32'), ('worldcup', 'http://www.wikidata.org/entity/Q160813')]
[('worldcupname', '1958 FIFA World Cup'), ('matches', '3

21

### Which world cup was cancelled? 

In [52]:
# which world cup was cancelled? 
queryString = """
SELECT *
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup ?p ?o .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "canc", "i") .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcup', 'http://www.wikidata.org/entity/Q203851'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q107745632'), ('worldcupname', '1942 FIFA World Cup'), ('oname', 'cancelled event due World War II')]
[('worldcup', 'http://www.wikidata.org/entity/Q65273310'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q107745632'), ('worldcupname', '1946 FIFA World Cup'), ('oname', 'cancelled event due World War II')]


2

## What was the duration of both world wars?


In [53]:
queryString = """
SELECT *
WHERE { 

    wd:Q203851 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('o', 'http://www.wikidata.org/entity/Q253414'), ('pname', 'organizer'), ('oname', 'FIFA')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q107745632'), ('pname', 'instance of'), ('oname', 'cancelled event due World War II')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('pname', 'instance of'), ('oname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('o', 'http://www.wikidata.org/entity/Q2736'), ('pname', 'sport'), ('oname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('o', 'http://www.wikidata.org/entity/Q31930761'), ('pname', 'competition class'), ('oname', "men's association football")]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('o', 'http://www.wikidata.org/entity/Q19317'), ('pname', 'sports season of league or competition'), ('oname', 'FIFA World Cup')]


6

In [54]:
queryString = """
SELECT *
WHERE { 

    ?p ?o wd:Q203851 .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q25536122'), ('o', 'http://www.w3.org/2002/07/owl#sameAs')]
[('p', 'http://www.wikidata.org/entity/Q19317'), ('o', 'http://www.wikidata.org/prop/direct/P527')]
[('p', 'https://www.wikidata.org/wiki/Special:EntityData/Q203851'), ('o', 'http://schema.org/about')]


3

In [55]:
# http://www.wikidata.org/entity/Q107745632 cancelled event due World War II
# [('p', 'http://www.wikidata.org/prop/direct/P31'), 
# ('o', 'http://www.wikidata.org/entity/Q107745632'), ('pname', 'instance of'), ('oname', 'cancelled event due World War II')]
queryString = """
SELECT *
WHERE { 

    wd:Q107745632 ?p ?o .
    # Labels
    #?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'http://www.wikidata.org/entity/Q362')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q107028274')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q107028274')]
[('p', 'http://www.wikidata.org/prop/direct/P828'), ('o', 'http://www.wikidata.org/entity/Q362')]
[('p', 'http://schema.org/name'), ('o', 'cancelled event due World War II')]


6

In [56]:

# http://www.wikidata.org/entity/Q107745632 cancelled event due World War II
queryString = """
SELECT *
WHERE { 

    wd:Q107745632 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'http://www.wikidata.org/entity/Q362'), ('pname', 'facet of'), ('oname', 'World War II')]
[('p', 'http://www.wikidata.org/prop/direct/P828'), ('o', 'http://www.wikidata.org/entity/Q362'), ('pname', 'has cause'), ('oname', 'World War II')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q107028274'), ('pname', 'subclass of'), ('oname', 'cancelled event edition')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q107028274'), ('pname', 'instance of'), ('oname', 'cancelled event edition')]


4

In [57]:
# [('p', 'http://www.wikidata.org/prop/direct/P828'),
#('o', 'http://www.wikidata.org/entity/Q362'), ('pname', 'has cause'), ('oname', 'World War II')]
queryString = """
SELECT *
WHERE { 

    wd:Q362 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "ww", "i") 
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1120'), ('o', '73000000'), ('pname', 'number of deaths')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('o', 'NQ 2545 - NQ 2795'), ('pname', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('o', 'http://www.wikidata.org/entity/Q3247957'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('o', '10644634'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0031535'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('o', 'event/World-War-II'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('o', 'http://www.wikidata.org/entity/Q5928186'), ('pname', "top

40

In [58]:
# [('p', 'http://www.wikidata.org/prop/direct/P828'),
#('o', 'http://www.wikidata.org/entity/Q362'), ('pname', 'has cause'), ('oname', 'World War II')]
queryString = """
SELECT *
WHERE { 

    wd:Q362 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
    #?o <http://schema.org/name> ?oname .
    #FILTER regex(?pname, "time", "i") 
    FILTER(REGEX(STR(?pname), "[Tt]ime"))
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3221'), ('o', 'subject/world-war-ii-193945'), ('pname', 'New York Times topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('o', '1939-09-01T00:00:00Z'), ('pname', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('o', '1945-09-02T00:00:00Z'), ('pname', 'end time')]


3

### World wars duration

In [59]:
# ('oname', 'World War II')]
queryString = """
SELECT *
WHERE { 

    wd:Q362 wdt:P580 ?starttime ;
            wdt:P582 ?endtime.
            
} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('starttime', '1939-09-01T00:00:00Z'), ('endtime', '1945-09-02T00:00:00Z')]


1

#### Who were the best players of every world cup? and where are they from?

In [60]:
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    wd:Q176883 ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    FILTER regex(?oname, "david", "i") .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3279'), ('o', 'http://www.wikidata.org/entity/Q83006'), ('pname', 'statistical leader'), ('oname', 'David Villa')]


1

In [61]:
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    wd:Q176883 wdt:P3279 ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    ?o <http://schema.org/name> ?oname .
    #?p <http://schema.org/name> ?pname .
    FILTER regex(?oname, "david", "i") .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q83006'), ('oname', 'David Villa')]


1

In [62]:
# Q83006
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    #wd:Q176883 wdt:P3279 ?ss .
    wd:Q83006 ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .

} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q937857'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q18543'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q324867'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q575453'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('o', 'n2015066305'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('o', '01000007332'), ('pname', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0520723'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('o', 'http://www.wikidata.org/entity/Q176883'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P

60

In [63]:
# Q83006
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    # wd:Q176883 wdt:P3279 ?o .
    wd:Q83006 ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('o', 'http://www.wikidata.org/entity/Q937857'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q18543'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q324867'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('o', 'http://www.wikidata.org/entity/Q575453'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('o', 'n2015066305'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('o', '01000007332'), ('pname', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('o', '0520723'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('o', 'http://www.wikidata.org/entity/Q176883'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P

10

In [64]:
# Q83006
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    # wd:Q176883 wdt:P3279 ?o .
    wd:Q83006 ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
    FILTER regex(?pname, "twitter", "i") .
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('o', 'Guaje7Villa'), ('pname', 'Twitter username')]


1

In [65]:
# Q83006
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    # wd:Q176883 wdt:P3279 ?o .
    wd:Q83006 ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
    FILTER regex(?pname, "follow", "i") .
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8687'), ('o', '9040925'), ('pname', 'social media followers')]


1

In [66]:
# Q83006 david villa social media followers
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    # wd:Q176883 wdt:P3279 ?o .
    wd:Q83006 wdt:P8687 ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?o <http://schema.org/name> ?oname .
    #?p <http://schema.org/name> ?pname .
    #FILTER regex(?pname, "follow", "i") .
} 
LIMIT 60
"""

print("Results")
run_query(queryString)

Results
[('o', '9040925')]


1

## Show the name of best players in every world cup.

In [67]:
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    ?worldcup wdt:P3279 ?players .
    ?players ?p ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    ?p <http://schema.org/name> ?pname .
    #FILTER regex(?oname, "david", "i") .

} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('worldcup', 'http://www.wikidata.org/entity/Q20738612'), ('players', 'http://www.wikidata.org/entity/Q542025'), ('p', 'http://www.wikidata.org/prop/direct/P3660'), ('o', '8171'), ('playersname', 'Aleksandr Kozlov'), ('pname', 'FootballFacts.ru player ID')]
[('worldcup', 'http://www.wikidata.org/entity/Q16636277'), ('players', 'http://www.wikidata.org/entity/Q726199'), ('p', 'http://www.wikidata.org/prop/direct/P3645'), ('o', '1031'), ('playersname', 'Dan Carter'), ('pname', 'All Blacks player ID')]
[('worldcup', 'http://www.wikidata.org/entity/Q18124957'), ('players', 'http://www.wikidata.org/entity/Q554866'), ('p', 'http://www.wikidata.org/prop/direct/P4050'), ('o', '4/de-colo-30234'), ('playersname', 'Nando de Colo'), ('pname', 'French Olympic Committee athlete ID')]
[('worldcup', 'http://www.wikidata.org/entity/Q20738612'), ('players', 'http://www.wikidata.org/entity/Q542025'), ('p', 'http://www.wikidata.org/prop/direct/P4421'), ('o', 'Futbol/Aleksandr-Sergeevich-Kozlov-Fu

20

In [68]:
queryString = """
SELECT *
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    ?worldcup wdt:P3279 ?players .
    ?players wdt:P8687 ?o .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    #?players <http://schema.org/name> ?playersname .
    #?p <http://schema.org/name> ?pname .
    #FILTER regex(?oname, "david", "i") .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcup', 'http://www.wikidata.org/entity/Q4629330'), ('players', 'http://www.wikidata.org/entity/Q16227014'), ('o', '61162')]
[('worldcup', 'http://www.wikidata.org/entity/Q18165874'), ('players', 'http://www.wikidata.org/entity/Q16228417'), ('o', '27323')]
[('worldcup', 'http://www.wikidata.org/entity/Q16192062'), ('players', 'http://www.wikidata.org/entity/Q16233299'), ('o', '34188')]
[('worldcup', 'http://www.wikidata.org/entity/Q647475'), ('players', 'http://www.wikidata.org/entity/Q45901'), ('o', '2314069')]
[('worldcup', 'http://www.wikidata.org/entity/Q2479190'), ('players', 'http://www.wikidata.org/entity/Q455122'), ('o', '1856')]
[('worldcup', 'http://www.wikidata.org/entity/Q4594671'), ('players', 'http://www.wikidata.org/entity/Q458576'), ('o', '28895')]
[('worldcup', 'http://www.wikidata.org/entity/Q166551'), ('players', 'http://www.wikidata.org/entity/Q178649'), ('o', '2538832')]
[('worldcup', 'http://www.wikidata.org/entity/Q494633'), ('players', 'http://www.

40

### The number of social media followers of players with the most goals in world cup 2010

In [69]:
# The number of social media followers of players with more goals in world cup 2010

queryString = """
SELECT ?playersname ?socialmediafollowers
WHERE { 
    #wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    wd:Q176883 wdt:P3279 ?players .
    ?players wdt:P8687 ?socialmediafollowers .
    # Labels
    #?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    #?p <http://schema.org/name> ?pname .
    #FILTER regex(?oname, "david", "i") .

} 
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('playersname', 'David Villa'), ('socialmediafollowers', '9040925')]
[('playersname', 'Diego Forlán'), ('socialmediafollowers', '5159742')]
[('playersname', 'Wesley Sneijder'), ('socialmediafollowers', '3992445')]
[('playersname', 'Thomas Müller'), ('socialmediafollowers', '4365133')]


4

### The name of players with the most goals in every world cup.

In [70]:
# The name of players with the most goals in every world cup.

queryString = """
SELECT ?worldcupname ?playersname 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    #?worldcup ?p ?o .
    ?worldcup wdt:P3279 ?players .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    #?p <http://schema.org/name> ?pname .

} 
ORDER BY DESC (?worldcupname)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2018 FIFA World Cup'), ('playersname', 'Harry Kane')]
[('worldcupname', '2014 FIFA World Cup'), ('playersname', 'Thomas Müller')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'David Villa')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Diego Forlán')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Wesley Sneijder')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Thomas Müller')]
[('worldcupname', '2006 FIFA World Cup'), ('playersname', 'Miroslav Klose')]
[('worldcupname', '2002 FIFA World Cup'), ('playersname', 'José Luis Chilavert')]
[('worldcupname', '1998 FIFA World Cup'), ('playersname', 'Davor Šuker')]
[('worldcupname', '1994 FIFA World Cup'), ('playersname', 'Hristo Stoichkov')]
[('worldcupname', '1994 FIFA World Cup'), ('playersname', 'Oleg Salenko')]
[('worldcupname', '1990 FIFA World Cup'), ('playersname', 'Salvatore Schillaci')]
[('worldcupname', '1986 FIFA World Cup'), ('playersname', 'Gary Lineker

30

In [71]:
# The name of players with the most goals in every world cup.

queryString = """
SELECT ?worldcupname ?playersname ?nationalityname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    # ?worldcup ?p ?o .
    ?worldcup wdt:P3279 ?players .
    ?players wdt:P27 ?nationality .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    #?players <http://schema.org/name> ?playersname .
    #?p <http://schema.org/name> ?pname .
    ?nationality <http://schema.org/name> ?nationalityname .
} 
ORDER BY DESC (?worldcupname)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2018 FIFA World Cup'), ('nationalityname', 'United Kingdom')]
[('worldcupname', '2014 FIFA World Cup'), ('nationalityname', 'Germany')]
[('worldcupname', '2010 FIFA World Cup'), ('nationalityname', 'Germany')]
[('worldcupname', '2010 FIFA World Cup'), ('nationalityname', 'Spain')]
[('worldcupname', '2010 FIFA World Cup'), ('nationalityname', 'Spain')]
[('worldcupname', '2010 FIFA World Cup'), ('nationalityname', 'Kingdom of the Netherlands')]
[('worldcupname', '2010 FIFA World Cup'), ('nationalityname', 'Uruguay')]
[('worldcupname', '2006 FIFA World Cup'), ('nationalityname', 'Germany')]
[('worldcupname', '2002 FIFA World Cup'), ('nationalityname', 'Paraguay')]
[('worldcupname', '1998 FIFA World Cup'), ('nationalityname', 'Croatia')]
[('worldcupname', '1998 FIFA World Cup'), ('nationalityname', 'Socialist Federal Republic of Yugoslavia')]
[('worldcupname', '1994 FIFA World Cup'), ('nationalityname', 'Soviet Union')]
[('worldcupname', '1994 FIFA World Cup'), (

34

In [72]:
# The name of players with the most goals in every world cup with the name of their country. 

queryString = """
SELECT ?worldcupname ?playersname ?countryname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    # ?worldcup ?p ?o .
    ?worldcup wdt:P3279 ?players .
    ?worldcup wdt:P1923 ?teams .
    #?teams wdt:P17 ?country .
    ?players wdt:P17 ?country .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    ?country <http://schema.org/name> ?countryname .

} 
ORDER BY DESC (?worldcupname)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [73]:
# The name of players with the most goals in every world cup with the name of their country. 

queryString = """
SELECT ?worldcupname ?teams
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P1923 ?teams .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?teams <http://schema.org/name> ?teamsname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '1938 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '2006 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '1982 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '1986 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '1978 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '2002 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '1974 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q166196')]
[('worldcupname', '1994 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q168397')]
[('worldcupname', '1938 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q168397')]
[('worldcupname', '1970 FIFA World Cup'), ('teams', 'http://www.wikidata.org/entity/Q168397')]


10

In [74]:
# The name of players with the most goals in every world cup with the name of their country. 

queryString = """
SELECT ?worldcupname ?playersname ?countryname
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3279 ?players .
    ?players wdt:P1532 ?country .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    ?country <http://schema.org/name> ?countryname .

} 
ORDER BY DESC (?worldcupname)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2018 FIFA World Cup'), ('playersname', 'Harry Kane'), ('countryname', 'England')]
[('worldcupname', '2014 FIFA World Cup'), ('playersname', 'Thomas Müller'), ('countryname', 'Germany')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'David Villa'), ('countryname', 'Spain')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Diego Forlán'), ('countryname', 'Uruguay')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Wesley Sneijder'), ('countryname', 'Netherlands')]
[('worldcupname', '2010 FIFA World Cup'), ('playersname', 'Thomas Müller'), ('countryname', 'Germany')]
[('worldcupname', '2006 FIFA World Cup'), ('playersname', 'Miroslav Klose'), ('countryname', 'Germany')]
[('worldcupname', '2002 FIFA World Cup'), ('playersname', 'José Luis Chilavert'), ('countryname', 'Paraguay')]
[('worldcupname', '1998 FIFA World Cup'), ('playersname', 'Davor Šuker'), ('countryname', 'Croatia')]
[('worldcupname', '1998 FIFA World Cup'), ('playersn

25

### The name of players with the most goals in every world cup with the name of their country. 

In [75]:
# The name of players with the most goals in every world cup with the name of their country. 

queryString = """
SELECT ?worldcupname (GROUP_CONCAT(DISTINCT ?playersname; separator=";") as ?players)
(GROUP_CONCAT(?countryname ; separator=";") as ?country ) 
WHERE { 
    wd:Q19317 wdt:P527 ?worldcup .
    ?worldcup wdt:P3279 ?players .
    ?players wdt:P1532 ?country .
    # Labels
    ?worldcup <http://schema.org/name> ?worldcupname .
    ?players <http://schema.org/name> ?playersname .
    ?country <http://schema.org/name> ?countryname .

} 
ORDER BY DESC (?worldcupname)
LIMIT 40
"""

print("Results")
run_query(queryString)

Results
[('worldcupname', '2018 FIFA World Cup'), ('players', 'Harry Kane'), ('country', 'England')]
[('worldcupname', '2014 FIFA World Cup'), ('players', 'Thomas Müller'), ('country', 'Germany')]
[('worldcupname', '2010 FIFA World Cup'), ('players', 'David Villa;Diego Forlán;Thomas Müller;Wesley Sneijder'), ('country', 'Spain;Uruguay;Netherlands;Germany')]
[('worldcupname', '2006 FIFA World Cup'), ('players', 'Miroslav Klose'), ('country', 'Germany')]
[('worldcupname', '2002 FIFA World Cup'), ('players', 'José Luis Chilavert'), ('country', 'Paraguay')]
[('worldcupname', '1998 FIFA World Cup'), ('players', 'Davor Šuker'), ('country', 'Croatia;Socialist Federal Republic of Yugoslavia')]
[('worldcupname', '1994 FIFA World Cup'), ('players', 'Hristo Stoichkov;Oleg Salenko'), ('country', 'Bulgaria;Russia;Soviet Union;Ukraine')]
[('worldcupname', '1990 FIFA World Cup'), ('players', 'Salvatore Schillaci'), ('country', 'Italy')]
[('worldcupname', '1986 FIFA World Cup'), ('players', 'Gary Line

16